In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import col, split, when


In [9]:
import findspark
findspark.init()

In [10]:
spark=SparkSession.builder.appName('BigData Cricket Analysis').getOrCreate()

In [11]:
spark

In [12]:
fact_balls = spark.read.csv("C:\\Users\\Patron\\Downloads\\balls.csv", header = True)
dim_matches = spark.read.csv("C:\\Users\\Patron\\Downloads\\matches.csv", header = True)
dim_players = spark.read.csv("C:\\Users\\Patron\\Downloads\\players.csv", header = True)

In [13]:
deliveries = fact_balls.select(
    col("match_id"),
    col("innings"),
    split(col("ball").cast("string"), "\.")[0].alias("overs"),
    split(col("ball").cast("string"), "\.")[1].alias("ball"),
    col("striker"),
    col("non_striker"),
    col("bowler"),
    col("runs_off_bat"),
    col("extras"),
    col("wides"),
    col("noballs"),
    col("byes"),
    col("legbyes"),
    col("penalty"),
    col("wicket_type"),
    col("player_dismissed"),
    col("other_wicket_type"),
    col("other_player_dismissed")
)

# Defining additional columns as per the provided SQL logic
result = deliveries.select(
    col("match_id"),
    col("striker"),
    col("non_striker"),
    col("bowler"),
    col("innings"),
    (col("overs").cast("integer") + 1).alias("overs"),
    col("ball").cast("integer").alias("ball"),
    col("runs_off_bat"),
    col("extras"),
    when(col("wides").isNotNull(), "WIDE")
        .when(col("noballs").isNotNull(), "NOBALL")
        .when(col("byes").isNotNull(), "BYE")
        .when(col("legbyes").isNotNull(), "LEGBYES")
        .when(col("penalty").isNotNull(), "PENALTY")
        .otherwise(None).alias("extras_type"),
    when(col("wicket_type").isNull(), "N").otherwise("Y").alias("wicket_flg"),
    col("wicket_type"),
    col("player_dismissed"),
    col("other_player_dismissed")
)

result.show()


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `wides` cannot be resolved. Did you mean one of the following? [`overs`, `ball`, `bowler`, `extras`, `innings`].;
'Project [match_id#128, innings#132, split(cast(ball#134 as string), \., -1)[0] AS overs#222, split(cast(ball#134 as string), \., -1)[1] AS ball#223, striker#129, non_striker#130, bowler#131, runs_off_bat#135, extras#136, 'wides, 'noballs, 'byes, 'legbyes, 'penalty, wicket_type#139, player_dismissed#140, 'other_wicket_type, other_player_dismissed#141]
+- Relation [match_id#128,striker#129,non_striker#130,bowler#131,innings#132,overs#133,ball#134,runs_off_bat#135,extras#136,extras_type#137,wicket_flg#138,wicket_type#139,player_dismissed#140,other_player_dismissed#141] csv
